In [ ]:
from collections import defaultdict
import requests
import re
from requests.exceptions import ReadTimeout
from requests.models import ReadTimeoutError
from urllib.parse import urlsplit
from collections import deque
from bs4 import BeautifulSoup
import pandas as pd
from googlesearch import search
from urllib.error import HTTPError

# Your file
filename = 'db.csv'
df = pd.read_csv(filename, sep = "\t", on_bad_lines='skip', encoding = 'utf-16')

# Your column name.
df["Full name"] = df["Full name"].apply(str.lower)
df["Full name"] = df["Full name"].str.replace(".","")

# Three new colums for the emails
df["M1"] = ""
df["M2"] = ""
df["M3"] = ""

# Reverse df. Erase if not needed.
df = df.iloc[::-1]

# The loop starts. 
for index, row in df.iterrows():
  urls = []
  original_url = urls
  unscraped = deque([original_url])
  scraped = set()
  emails = set()

  # Again, your row name and more stirngs if needed
  author_search = row["Full name"] + " contact"

  author = row["Full name"]

  # Splitting for comparing results latter on.
  author_sep = author.split()
  
  # Number of results
  urls = list(search(author_search, tld="com", num=10, stop=10, pause=2))
  count = defaultdict(int)

  # URL loop for first name search
  for i in urls:
    unscraped = deque([i])
    while len(unscraped):
        url = unscraped.popleft()  
        scraped.add(url)

        parts = urlsplit(url)
            
        base_url = "{0.scheme}://{0.netloc}".format(parts)
        if '/' in parts.path:
          path = url[:url.rfind('/')+1]
        else:
          path = url

        print("Crawling URL %s" % url)

        # The try/except following need fine-tuning
        try:
            response = requests.get(url, allow_redirects=False, timeout = 5)
        except HTTPError: # this need review. Pretty sure this is not good.
          pass
        except ReadTimeout:
          pass
        except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
            continue

        new_emails = set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.com", response.text, re.I))
        emails.update(new_emails) 

        soup = BeautifulSoup(response.text, 'lxml')

        for anchor in soup.find_all("a"):
          if "href" in anchor.attrs:
            link = anchor.attrs["href"]
          else:
            link = ''

            if link.startswith('/'):
                link = base_url + link
            
            elif not link.startswith('http'):
                link = path + link

            if not link.endswith(".gz"):
              if not link in unscraped and not link in scraped:
                  unscraped.append(link)

  lst_emails = list(emails)

  # Counting if substring in string
  for i in author_sep:
    for j in lst_emails:
      if i in j:
        count[j] = count[j] + len(i) 
  
  # Selecting 3 top results
  sorted_count = sorted(count, key=count.get, reverse = True)[:3]

  # To the df
  try:
    df.loc[index,"M1"] = sorted_count[0]
  except:
    pass

  try:
    df.loc[index, "M2"] = sorted_count[1]
  except:
    pass

  try:
    df.loc[index, "M3"] = sorted_count[2]
  except:
    pass

# Final export of new .csv file
df.to_csv('db_with_mails.csv', index=False)